In [45]:
# 104_job_bank with Selenium (Overview)
# import packages
from selenium.webdriver import Chrome
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import csv , time

# Get Driver
service = Service("./chromedriver")
chrome_options = Options()
chrome_options.page_load_strategy = 'eager'
driver = Chrome(service=service,options = chrome_options)


In [46]:
# 104 home page's url
url = "https://www.104.com.tw/jobs/main/"  # go to jobbank
driver.get(url) # surf jobbank with browser
driver.maximize_window() # maximize broswer window -> get more HTML ASAP
time.sleep(3)

In [47]:
# KeyWord of interest
driver.find_element(by=By.XPATH, value='/html/body/article[1]/div/div/div[4]/div/input').send_keys("後端工程師")
time.sleep(3)

In [48]:
# Push search button
driver.find_element(by=By.XPATH,value='/html/body/article[1]/div/div/div[4]/div/button').click()
time.sleep(3)

In [49]:
# find HTML element of jobs' info
# articles = driver.find_elements(By.CLASS_NAME, "b-block--top-bord.job-list-item.b-clearfix.js-job-item")
jobs = driver.find_elements(By.CLASS_NAME, "js-job-link")
jobs_numbers = len(jobs)

In [50]:
# Scroll action
# Step 1 : get all pages' count
button = driver.find_element(By.CLASS_NAME,"b-clear-border.page-select.js-paging-select.gtm-paging-bottom")
Total_pages_count = len(button.text.split("\n"))

# Step 2 : auto scroll + manual click
manual_scroll_count = 1
while manual_scroll_count :
    if manual_scroll_count + 15 == Total_pages_count:
        print("================ 已到達最底，翻頁完成 !!=======================")
        break
    driver.execute_script("window.scrollTo(0,document.body.scrollHeight)")
    time.sleep(1)
    try :
        button = driver.find_element(By.CLASS_NAME,"b-btn.b-btn--link.js-more-page")
        driver.execute_script("arguments[0].click();", button)
        print(f"================需要手動!! 按一下按鈕! -> 目前翻到第{manual_scroll_count + 15}頁=================")
        manual_scroll_count += 1
        time.sleep(2)
    except Exception as e:
        print("================不用手動，繼續滑下去!! 直到翻到第15頁=================")

================不用手動，繼續滑下去!! 直到翻到第15頁=================
================不用手動，繼續滑下去!! 直到翻到第15頁=================
================不用手動，繼續滑下去!! 直到翻到第15頁=================
================不用手動，繼續滑下去!! 直到翻到第15頁=================
================不用手動，繼續滑下去!! 直到翻到第15頁=================
================不用手動，繼續滑下去!! 直到翻到第15頁=================
================不用手動，繼續滑下去!! 直到翻到第15頁=================
================不用手動，繼續滑下去!! 直到翻到第15頁=================
================不用手動，繼續滑下去!! 直到翻到第15頁=================
================不用手動，繼續滑下去!! 直到翻到第15頁=================
================不用手動，繼續滑下去!! 直到翻到第15頁=================
================不用手動，繼續滑下去!! 直到翻到第15頁=================
================不用手動，繼續滑下去!! 直到翻到第15頁=================
================需要手動!! 按一下按鈕! -> 目前翻到第16頁=================
================需要手動!! 按一下按鈕! -> 目前翻到第17頁=================
================需要手動!! 按一下按鈕! -> 目前翻到第18頁=================
================需要手動!! 按一下按鈕! -> 目前翻到第19頁=================
================需要手動!! 按一下按鈕! -> 目前翻到第20頁========

In [53]:
job_name = driver.find_elements(By.CLASS_NAME, "js-job-link")
len(job_name)

2990

In [9]:
# find job details in articles which has same form individually
for nth_job in range(jobs_numbers) :
    # articles = driver.find_elements(By.CLASS_NAME, "b-block--top-bord.job-list-item.b-clearfix.js-job-item")
    # find nth-job's link
    job_name = driver.find_elements(By.CLASS_NAME, "js-job-link")[nth_job]
    # get value of attribute (key) in every element (ex: "href" = "url" 、 "class" = "js-job-link")
    job_link = job_name.get_attribute("href")
    print(job_name.text)
    print(job_link)
    driver.get(job_link)
    time.sleep(3)
    driver.back()
    time.sleep(5)

大流量社群平台資深後端工程師 /Senior Backend Engineer(Golang)
https://www.104.com.tw/job/5bdsg?jobsource=hotjob_chr
PHP Laravel工程師(中階以上)
https://www.104.com.tw/job/7p2s1?jobsource=hotjob_chr
軟體專案管理人員
https://www.104.com.tw/job/74y1s?jobsource=hotjob_chr
Frontend / Computer Vision / Golang / iOS Senior Engineer
https://www.104.com.tw/job/7tz7f?jobsource=hotjob_chr
(總公司-資訊部)後端工程師
https://www.104.com.tw/job/75ytw?jobsource=hotjob_chr
前端與後端技術工程師 (具三年以上經驗)
https://www.104.com.tw/job/7kuen?jobsource=hotjob_chr
高級後端工程師**月薪60,000~80,000**
https://www.104.com.tw/job/74xzl?jobsource=hotjob_chr
資深網站前端工程師 (Senior Frontend Developer)
https://www.104.com.tw/job/5udee?jobsource=hotjob_chr
資深後端工程師**月薪80,000~100,000**
https://www.104.com.tw/job/74y02?jobsource=hotjob_chr
PHP 或 Java後端工程師 (視訊/線上面談)-擴編職缺
https://www.104.com.tw/job/64v7o?jobsource=hotjob_chr
Backend Engineer 後端工程師
https://www.104.com.tw/job/7nq2e?jobsource=jolist_b_relevance
Backend Engineer後端工程師
https://www.104.com.tw/job/5bwie?jobsource=jolist_b_relev

In [10]:
jobs_numbers

30

In [11]:
job_name = driver.find_elements(By.CLASS_NAME, "js-job-link")[29]

In [12]:
job_name.text

'RD-後端工程師 Backend Engineer'